# GPU Programming Workshop
### April 19, 2023

## Simple Example
We'll begin with a very simple example: adding two vectors a and b. We'll do it in two ways: first in "normal" (ie, cpu) python, then with a gpu.

In [12]:
# numpy is the python array library
import numpy as np
# we'll compare how long the cpu/gpu takes
import time

In [22]:
# create a rng (random number generator)
rng = np.random.default_rng()

# make two arrays, a and b, that we want to add, with random numbers from 1 to 9
n_elements = 10_000_000
a = rng.integers(0, 9, n_elements)
b = rng.integers(0, 9, n_elements)

# c will be the result of a + b
c = np.empty(n_elements)

##### Write code here set c = a + b:

In [23]:
start = time.time()
for index in range(n_elements):
    c[index] = a[index] + b[index]
elapsed = time.time() - start

print("Took", elapsed, "seconds")
print(c)
assert (c == a + b).all(), "Addition failed!"

Took 3.1232004165649414 seconds
[8. 9. 5. ... 6. 9. 9.]


### GPU Speedup
For 10 million elements, this already takes a few seconds. Let's speed that up with a gpu!

In [24]:
# cuda is the gpu library
from numba import cuda
import math
import numpy as np
import time

In [25]:
# reset c
c = np.empty(n_elements)

##### Write gpu code here set c = a + b:

In [26]:
# to run code on a gpu, we have to write an entire function to give to the gpu
@cuda.jit
def add_arrays_gpu(a, b, c):
    index = cuda.grid(1)
    c[index] = a[index] + b[index]

threads_per_block = 64
# make enough blocks so that each element in a & b can get added
blocks = math.ceil(n_elements // threads_per_block)

start = time.time()
add_arrays_gpu[blocks, threads_per_block](a, b, c)
elapsed = time.time() - start

print("Took", elapsed, "seconds")
print(c)

Took 0.16608667373657227 seconds
[8. 9. 5. ... 6. 9. 9.]


/home/aschwart/anaconda3/envs/gpu2/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


## Calculating π
We'll do two versions again, to show the power of the gpu!

In [27]:
# for the correct value of π
import math
# see how long the calculation takes
import time
# array calculation library
import numpy as np
# create plot!
import matplotlib.pyplot as plt

In [47]:
# initialize our random number generator
rng = np.random.default_rng()

def calc_pi(n_darts, plot=False):
    x = rng.uniform(0, 1, n_darts)
    y = rng.uniform(0, 1, n_darts)
    r = np.sqrt(x ** 2 + y ** 2)
    if plot:
        plt.scatter(x[r < 1], y[r < 1])
        plt.show()
    return 4 * np.sum(r < 1) / n_darts

num_darts = 2_000_000
# num_darts = 20_000_000
# num_darts = 200_000_000
# num_darts = 2_000_000_000

start = time.time()
pi_est = calc_pi(num_darts, plot=False)
elapsed = time.time() - start

print("Took", elapsed, "seconds")
print("Estimated pi =", pi_est)

Took 0.04975295066833496 seconds
Estimated pi = 3.142394


### Now let's calculate π again, with a GPU!

In [37]:
# random number functions
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
import math
from numba import cuda
import time
# import matplotlib.pyplot as plt

In [49]:
@cuda.jit
def calc_pi_gpu(rng, iterations, totals):
    id = cuda.grid(1)
    inside = 0
    for i in range(iterations):
        x = xoroshiro128p_uniform_float32(rng, id)
        y = xoroshiro128p_uniform_float32(rng, id)
        r = math.sqrt(x ** 2 + y ** 2)
        inside += r < 1

    totals[id] = 4 * inside / iterations

# 1000 iters is about 8 million darts
threads_per_block = 64
blocks = 128
iterations = num_darts // (blocks * threads_per_block)

totals = np.zeros(blocks * threads_per_block)

seed = np.random.random(1)[0]
rngs = create_xoroshiro128p_states(blocks * threads_per_block, seed=seed)

start = time.time()
calc_pi_gpu[blocks, threads_per_block](rngs, iterations, totals)
elapsed = time.time() - start

print("Took", elapsed, "seconds")
print("Estimated pi =", totals.mean())

Took 0.14090657234191895 seconds
Estimated pi = 3.1415515336834012


/home/aschwart/anaconda3/envs/gpu2/lib/python3.10/site-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
